# Climate Analysis

In [2]:
# import dependancies
from sqlalchemy import create_engine, Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import matplotlib.pyplot as plt
import pandas
from flask import jsonify

In [ ]:
Base = declarative_base()
engine = create_engine('sqlite:///hawaii.sqlite')
session = Session(bind=engine)